# Introduction to metaprogramming: "Code that creates code" 

Julia has strong **metaprogramming** capabilities. What does this mean?

> **meta**: something on a higher level

**metaprogramming** = "higher-level programming"

i.e. writing code (a program) to manipulate not data, but code (that itself manipulates data)


## Motivating example

Metaprogramming has many different uses, several of which we will explore. It is often a way to add, or change, the syntax of Julia, to write a "domain-specific language" (DSL), that converts a simple syntax (but that is not standard Julia syntax) into true Julia code.

As a motivating example, we might like to be able to write

In [ ]:
∑_{i ≠ j} 1/(λ_i - λ_j)  # NOT JULIA SYNTAX!

and have it *automatically converted* into something like

In [ ]:
total = zero(λ[1])

for i in 1:N
    j == i && continue
    
    total += 1 / (λ[i] - λ[j])
end

total

## A simpler example 

This is too hard [**exercise**: write this functionality and make it into a package!], let's start out with a simpler example: Wilkinson-type polynomials.
The [Wilkinson polynomial](https://en.wikipedia.org/wiki/Wilkinson's_polynomial) is

$$p_{20}(x) := (x-1) \cdot (x-2) \cdot \cdots \cdot (x-20) = \prod_{i=1}^{20} (x-i).$$

[Polynomials like this are interesting, since the eigenvalues of the associated "companion matrix", which are used to find the roots of the polynomial, are very sensitive to perturbations of the coefficients of the polynomial.]

Suppose we wish to define this polynomial in Julia. The simple way would be to write it out explicitly:

In [ ]:
p_5(x) = (x-1) * (x-2) * (x-3) * (x-4) * (x-5)

$p_{10}$ is already a pain to type by hand, $p_{20}$ more so, and $p_{100}$ is basically impossible. But this is just a case of repetition, and computers are designed for that. One possible definition uses a `for` loop:

In [ ]:
function wilkinson(n, x)
    result = x - 1
    
    for i in 2:n
        result *= (x - i)
    end
    
    result
end

We can use an anonymous function to define the function $p_n$:

In [ ]:
p(n) = x -> wilkinson(n, x)

In [ ]:
p(4)(3.1)

[In Julia 0.4, anonymous functions have a performance penalty, although they no longer do in 0.5.]

It seems, though, that it should be possible to use the original definition of the function to write the equivalent of

In [ ]:
p_5(x) = (x-1)*(x-2)*(x-3)*(x-4)*(x-5)

for $n=100$.
In other languages, this would often be accomplished by manipulating strings that represent the "surface syntax", i.e. the string of characters that you would actually type, and then evaluate this string. However, in Julia
**we never use strings for this**, since Julia has a way to **refer to Julia code objects within Julia**.

## Expressions 

Let's take the simplest polynomial, that we write as `(x-1) * (x-2)`. We can view this as a piece of Julia code, called an **expression**, and can tell Julia this as follows:

In [ ]:
ex = quote   
        (x - 1) * (x - 2)
    end

For small pieces of code like this, an alternative syntax uses the `:( ... )`  operator:

In [ ]:
ex = :( (x-1) * (x-2) )

What does Julia think this is?

In [ ]:
typeof(ex)

We see that Julia has a type `Expr` that represents an **expression object**, that we can think of as a "Julia code snippet".

We would like to be able to execute this code; we can do so with `eval` ("evaluate"):

In [ ]:
eval(ex)

We see that `x` is not defined, since Julia is trying to do the same as if we had written the following straight at the prompt

In [ ]:
(x-1) * (x-2)

If we give `x` a value first, then it works:

In [ ]:
x = 3.5
(x-1) * (x-2)

And so hence does the `eval`:

In [ ]:
eval(ex)

For example, we can try to write a simple formula evaluator:

In [ ]:
formula = "3x"

In [ ]:
eval(formula)

This does not do what we expect, since we have a string, not a Julia expression:

In [ ]:
typeof(formula)

To convert this string into an expression, we use `parse`:

In [ ]:
formula2 = parse(formula)

In [ ]:
eval(formula2)

Note that the really hard work, that of parsing the expression, i.e. converting it into an internal representation in terms of a tree, has already been done for us by Julia. It is thus, happily, usually not necessary for us to write our own parser; we just leverage Julia's own! This is one of many ways in which Julia minimises the work that we need to do, compared to other languages.

## Structure of `Expr`essions 

An expression object of type `Expr` has an internal structure that corresponds to the **abstract syntax tree** (AST) of the expression, i.e. a tree, whose nodes are operations, and whose children are subtrees.
We can see this in two ways, using `dump`:

In [ ]:
dump(ex)

which shows everything [up to a pre-determined depth] in detail, or 

In [ ]:
Meta.show_sexpr(ex)

which gives a compact version (similar to an [S-expression](https://en.wikipedia.org/wiki/S-expression) in Lisp, whence the name of the function, which is in the `Meta` submodule in `Base`). They both give representations of the syntax tree describing the hierarchical structure of the expression.

The point is that since an `Expr`ession is a **standard Julia objects**, we can **use standard Julia commands to manipulate it**! 

As usual, doing `ex.<TAB>` gives a list of the fields in the type `Expr`, or we can use `fieldnames(ex)`:

In [ ]:
fieldnames(ex)

and then examine the fields:

In [ ]:
ex.head

This tells us that the expression represents a function call, whose arguments are the following:

In [ ]:
ex.args

[`ex.typ` is not useful for the user.]

The first element of the array `ex.args` is the function to be called:

In [ ]:
ex.args[1]

and the second element is

In [ ]:
ex.args[2]

that is, it is itself an `Expr`ession:

In [ ]:
typeof(ans)

Thus the structure of `Expr`essions is fundamentally recursive, and so lends itself naturally to recursive algorithms.

We can dive, in turn, into the structure of each element:

In [ ]:
ex.args[2].args

In [ ]:
ex.args[2].args[2]

What is this `:x`?

In [ ]:
typeof(ex.args[2].args[2])

We see that it has a special `Symbol` type, which represents the atoms (smallest parts) of an expression.

## Modifying expressions 

Now, what happens if we **modify** this?  Let's make a copy of the expression first so that we can compare the two.

In [ ]:
ex2 = copy(ex)

In [ ]:
ex2.args[2].args[2] = :z

We have changed something inside the object `ex2`, so let's look at it:

In [ ]:
ex2

In [ ]:
ex

The original expression has, indeed, changed! That is, we have taken a piece of Julia code, and used Julia itself to manipulate it into a different piece of Julia code. This is one of the simplest examples of metaprogramming.

Now we can define `x` and `z` and evaluate the expressions:

In [ ]:
x = 3.5; z = 4.5

In [ ]:
eval(ex), eval(new_ex)

## Walking a syntax tree 

What if we wanted to replace *all* of the `x`s in a given expression. The problem is that they may be buried arbitrarily deeply in the nested syntax tree. So we have to traverse the tree, in order to visit each piece of it and check if it is an `:x`.

**Exercise**: Write a function that takes an expression object and replaces **all** of the `:x`s by `:z`s.

In [ ]:
function traverse!(ex::Expr)
    for (i, arg) in enumerate(ex.args)
        
        if arg == :x
            ex.args[i] = :z   # we cannot use arg here, since it is a copy, not a reference
        end
        
        if isa(arg, Expr)
            traverse!(arg)
        end
    end
end

In [ ]:
ex = :((x-1)*(x-2))

In [ ]:
traverse!(ex)
ex

Of course, we can now make this potentially more useful by generalising the function, allowing us to replace one sub-expression by another:

In [ ]:
function traverse!(ex::Expr, find, replace)
    for (i, arg) in enumerate(ex.args)
        
        if arg == find
            ex.args[i] = replace   # we cannot use arg here, since it is a copy, not a reference
        end
        
        if isa(arg, Expr)
            traverse!(arg)   # recursive
        end
    end
    
    ex
end

In [ ]:
traverse!( :( (x-1) * (x-2) ), :(x-1), :(x-10) )

In [ ]:
traverse!( :( (x-1) * (x-2) ), :(x), :(z) )

This is, of course, not general enough - for example, we cannot replace something of the form `:(x - a)` with `:(x - 2a)` with the current version; this would require more sophisticated pattern matching capabilities - but it demonstrates the basic idea.

#### Exercise
Julia by default uses standard 64-bit (or 32-bit) integers, which leads to surprising behaviour, e.g.

In [ ]:
2^60 * 2^60

No warning is given that there was an overflow in this calculation. 

However, in `Base` there are *checked* operations, such as `checked_mul`, which do throw an exception on overflow:

In [ ]:
Base.checked_mul(2^60, 2^60)

#### Exercise
Write a function `checked` that replaces standard functions (`-`, `+`, `*`, `/`) in an expression by their corresponding checked counterparts.

## Code generation

Let's return to the original question: how to create a long polynomial expression. 
So far, we have not seen how to *add* code, only *change* code that is already there. 

A natural idea is to build the code up step by step; this is known as **code generation**.

Let's start again from the first term:

In [ ]:
ex = :(x-1)

We would like to take what we have and create code that is "what we have multiplied by `:(x-2)`". Let's try:

In [ ]:
ex_new = :(ex * (x-2))

This doesn't work, since `ex` is treated as a symbol, whereas we need the *value* contained in the *variable* called `ex`. This is obtained using the `$` operator, a procedure called **interpolation**. (Compare this to string interpolation.)

In [ ]:
ex = :( $ex * (x-2) )

Now we can continue:

In [ ]:
ex = :( $ex * (x-3) ) 

Finally we see how to construct our loop:

In [ ]:
n = 10
ex = :(x-1)

for i in 2:n
    ex = :( $ex * (x-i) )
end

In [ ]:
ex

This did not work, since once again we did not want "the code '`i`'", but rather the value of the variable `i`. So:

In [ ]:
n = 10
ex = :(x-1)

for i in 2:n
    ex = :( $ex * (x - $i) )
end

In [ ]:
ex

This is almost what we would write by hand, except for the large number of parentheses.

Now we need to produce the name of the function:

In [ ]:
name = symbol("p_", n)   # use `Symbol` instead of `symbol` in Julia v0.5

The code is then

In [ ]:
code = :( $name(x) = $ex )

We can wrap this in a function:

In [ ]:
function make_wilkinson(n)
    
    ex = :(x-1)

    for i in 2:n
        ex = :( $ex * (x - $i) )
    end
    
    name = symbol("p_", n) 
    code = :( $name(x) = $ex )
    
    eval(code)
end

Finally we evaluate this:

In [ ]:
make_wilkinson(10)

In [ ]:
make_wilkinson(100)

This creates a function with the name `p_10` that does what we would write by hand.

Let's compare the two options by evaluating the function on a grid of values:

In [ ]:
function f1(range)
    total = 0.0
    for x in range
        total += p_100(x)
    end
    return total
end

function f2(range)
    total = 0.0
    for x in range
        total += wilkinson(100, x)
    end
    return total
end

In [ ]:
range = -10:0.000001:10
t1 = @elapsed f1(range);
t2 = @elapsed f2(range);
t2 / t1

We see that the generated code with the unrolled loop is 10% faster than the naive loop with 100 terms.

### Starting from empty

In some cases, it is useful to start from something empty and add statements to it.
To do this, we can do, for example,

In [ ]:
code = quote end   # empty

In [ ]:
dump(code)

[Note that in Julia 0.5, this is not "empty", since it has a `:line` line-number node.]

We can add statements using the standard Julia `push!`:

In [ ]:
Base.push!(code.args, :(a = 1))
Base.push!(code.args, :(b = a + 1))
code

In [ ]:
eval(code)

**Exercise**: Build up the Wilkinson example using this technique.

## Repetitive code

Code generation is used frequently in Julia code when repetitive code is required. For example, let's return to the idea of wrapping a type:

In [ ]:
type OurFloat
    x::Float64
end

We can generate objects of this type:

In [ ]:
a = OurFloat(3)
b = OurFloat(4)

But arithmetic operations are not defined:

In [ ]:
a + b

We can define them in the natural way:

In [ ]:
import Base: +, -, *, /
+(a::OurFloat, b::OurFloat) = a.x + b.x
-(a::OurFloat, b::OurFloat) = a.x - b.x

But this will quickly get dull, and we could easily make a mistake. 
As usual, whenever we are repeating something more than twice, we should try to automate it.

We have some code of the form

    *op*(a::OurFloat, b::OurFloat) = a.x *op* b.x
    
where `*op*` is supposed to represent the operator. Julia allows us to do this almost literally; we just need to substitute in the *value* of the *variable* `op`! This is an **exercise**

In [ ]:
for op in (:+, :-, :*, :/)
    @show op
    ex = :( $(op)(a::OurFloat, b::OurFloat) = $(op)(a.x, b.x) )
    @show ex
end

Finally we need to evaluate the code. The combination of `eval` and `:(...)` that we used above can be abbreviated to `@eval`:

In [ ]:
for op in (:+, :-, :*, :/)
    @eval $(op)(a::OurFloat, b::OurFloat) = $(op)(a.x, b.x) 
end

In [ ]:
a*b

In [ ]:
@which a*b

## Macros

It is very common in Julia to use things that look like functions but whose names start with `@`, e.g. `@time`, `@which`, etc. These are not functions in the standard sense, but rather are a kind of "super-function", called **macros**. 

A macro takes a **piece of Julia code** (`Expr`ession object) as its argument, manipulates that code to turn it into a new one, and **returns a new piece of code** as its output. The effect of a macro call is to insert that new piece of code in place of the old code, which is consequently compiled by the Julia compiler. 

Note that the user *does not need to explicitly pass an `Expr`ession object, since this is done automatically*.

To see this, let's define a simple macro:

In [ ]:
macro simple(expr)
    @show expr
    nothing   # return nothing for the moment
end

In [ ]:
@simple x+y

We see that the Julia code that follows the macro call is passed to the macro *already having been parsed into an `Expr` object*.

Suppose we redefine the macro as

In [ ]:
macro simple(expr)
    @show expr
    expr  # returns expr
end

Then we get

In [ ]:
@simple x+y

What is happening here is that the macro returns the expression `:(x+y)`, and this is then evaluated using `eval`.
The result is that Julia tries to calculate the value of the expression `x+y`, but the variable `x` is not defined. Let's define `y` and `z`, but not `x`:

In [ ]:
y = 3; z = 4

In [ ]:
x

In [ ]:
@simple x+y

Now let's define a new macro `@replace` that uses our previous `replace` function:

In [ ]:
macro traverse(expr)
    traverse!(expr)
    @show expr
    Meta.quot(expr)
end

In [ ]:
@traverse x+y

## `macroexpand`

We can discover what a macro does using the `macroexpand` function which takes a code expression:

In [ ]:
macroexpand(:(@time sin(10)))

As an example of the use of a macro, let's look at the `@interval` macro from the [`ValidatedNumerics.jl` package](https://github.com/dpsanders/ValidatedNumerics.jl):

In [ ]:
# Pkg.add("ValidatedNumerics")

In [ ]:
using ValidatedNumerics

In [ ]:
@interval(0.1)

Floating-point calculations are not precise. Interval arithmetic is a way to provide a guaranteed *enclosure* of a result, i.e. an interval that contains the true result. 

In [ ]:
macroexpand(:(@interval(0.1)))

In [ ]:
@interval sin(0.1) + cos(0.2)

In [ ]:
macroexpand(:(@interval sin(0.1) + cos(0.2)))

This has the structure `sin(interval(0.1)) + cos(interval(0.2))` (although the details are a bit more complicated).

So basically the `@interval` macro does something very similar to our `replace!` macro: it walks the tree of the expression; finds parts of it that are of a certain type (numeric literals); and wraps those in the `make_interval` function

## Exercise 

Define a `@checked` macro that uses the `check_arithmetic` function from a previous exercise to wrap convert a piece of code that uses arithmetic operations into the corresponding code that uses the checked versions instead.

## Macro hygiene

An important, but slippery topic with macros is so-called "hygiene". This refers to where variables are considered to be defined: inside the macro itself, or where the code that the macro creates is evaluated. We unfortunately do not have time to give this topic a detailed discussion. 

Basically, we need to allow variables to "escape" from the environment of the macro into the surrounding code, when the code returned by a macro refers to these external variables, rather than variables internal to the macro. This is done using `esc`; see the corresponding [Julia documentation](http://docs.julialang.org/en/release-0.4/manual/metaprogramming/#hygiene), and perhaps books on Lisp.